In [1]:
import re
import torch
from torch import nn
from torch.nn import functional as F
from torch.utils.data import DataLoader
from torch.utils.data import random_split
import pytorch_lightning as pl
from transformers import BertTokenizer, BertForMaskedLM
import random
from string import ascii_letters
from datasets import Dataset
from pytorch_lightning.loggers import TensorBoardLogger

/home/ogu/source/experiments/spelling_bert/venv/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from lightning_bert import *

In [3]:
def map_text(items):
    result = tokenizer(list(
        map(replace_augment, items["input"])), truncation=True, padding="max_length")
        
    target = tokenizer(
        items['input'], truncation=True, padding="max_length")
    result['target_ids'] = target["input_ids"]
    result['target_attention_mask'] = target["attention_mask"]

    return result

In [4]:
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

texts = load_data("en.txt")
texts = [text.lower() for text in texts]
my_dict = {"input":texts}
dataset = Dataset.from_dict(my_dict)
dataset = dataset.map(map_text, batched=True,num_proc=4)
dataset = dataset.remove_columns("input")
dataset.set_format(type='torch', columns=['input_ids', 'token_type_ids', 'attention_mask', 'target_ids', 'target_attention_mask'])

dataset = dataset.train_test_split(test_size=0.2)

train_dataloader = DataLoader(dataset["train"], shuffle=True, batch_size=8)
test_dataloader = DataLoader(dataset["test"], batch_size=16)



#0:   0%|          | 0/3 [00:00<?, ?ba/s]




#0:  33%|███▎      | 1/3 [00:00<00:01,  1.24ba/s]



#2: 100%|██████████| 3/3 [00:01<00:00,  1.54ba/s]


#0: 100%|██████████| 3/3 [00:01<00:00,  1.52ba/s]


In [5]:
# model
model = SpellingBert()

#logger

logger = TensorBoardLogger("tb_logs", name="my_model")
# training
trainer = pl.Trainer(logger=logger,gpus=1, precision=16, max_epochs=2)# ,limit_train_batches=0.5
trainer.fit(model, train_dataloader, test_dataloader)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Using 16bit native Automatic Mixed Precision (AMP)
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type             | Params
-------------------------------------------
0 | model | BertForMaskedLM  | 109 M 
1 | loss

Sanity Checking DataLoader 0:  50%|█████     | 1/2 [00:00<00:00,  6.80it/s]

/home/ogu/source/experiments/spelling_bert/venv/lib/python3.8/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:240: PossibleUserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 16 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
/home/ogu/source/experiments/spelling_bert/venv/lib/python3.8/site-packages/torch/optim/lr_scheduler.py:131: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn("Detected call of `lr_scheduler.step()` before `optimizer.s

/home/ogu/source/experiments/spelling_bert/venv/lib/python3.8/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:240: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 16 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Epoch 1: 100%|██████████| 1125/1125 [03:58<00:00,  4.71it/s, loss=2.02, v_num=21]


In [17]:
#sanity check

inputs = tokenizer("my nahme is peter", return_tensors="pt")

with torch.no_grad():
    logits = model.model(**inputs).logits

predicted_token_ids = logits.argmax(axis=-1)
print(tokenizer.decode(predicted_token_ids[0]))

[CLS] my family is is. [SEP]


In [7]:
a = tokenizer("my name is slim", return_tensors="pt",add_special_tokens=True)#
a

{'input_ids': tensor([[  101,  2026,  2171,  2003, 11754,   102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1]])}

In [8]:
tokenizer.decode(a['input_ids'][0])

'[CLS] my name is slim [SEP]'

In [9]:
tokenizer("my name is slim", return_tensors="pt", add_special_tokens=False)

{'input_ids': tensor([[ 2026,  2171,  2003, 11754]]), 'token_type_ids': tensor([[0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1]])}

In [ ]:
#facebook/bart-base

In [24]:
!python run_summarization.py \
    --model_name_or_path vinai/bartpho-word-base \
    --do_train \
    --do_eval \
    --evaluation_strategy="steps" \
    --eval_steps=5000 \
    --train_file vi.train.csv \
    --validation_file vi.test.csv \
    --output_dir ./models/test_model/ \
    --overwrite_output_dir \
    --per_device_train_batch_size=1 \
    --per_device_eval_batch_size=1 \
    --gradient_accumulation_steps=1 \
    --num_train_epochs=2 \
    --predict_with_generate \
	--logging_steps="10" \
    --save_total_limit="2" \
    --max_target_length=128 \
    --max_source_length=128 \
    --learning_rate="1e-6" \
    --use_mps_device \
    #--fp16

02/25/2023 08:09:57 - WARNING - __main__ - Process rank: -1, device: mps, n_gpu: 1distributed training: False, 16-bits training: False
02/25/2023 08:09:57 - INFO - __main__ - Training/evaluation parameters Seq2SeqTrainingArguments(
_n_gpu=1,
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
auto_find_batch_size=False,
bf16=False,
bf16_full_eval=False,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_pin_memory=True,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
ddp_timeout=1800,
debug=[],
deepspeed=None,
disable_tqdm=False,
do_eval=True,
do_predict=False,
do_train=True,
eval_accumulation_steps=None,
eval_delay=0,
eval_steps=5000,
evaluation_strategy=steps,
fp16=False,
fp16_backend=auto,
fp16_full_eval=False,
fp16_opt_level=O1,
fsdp=[],
fsdp_min_num_params=0,
fsdp_transformer_layer_cls_to_wrap=None,
full_determinism=False,
generation_max_length=None,
generation_num_beams=None,
gradient_accumulation_steps=1,
gradient_checkpo

In [12]:
!pip install lion_pytorch transformers datasets evaluate pip install jiwer